<a href="https://colab.research.google.com/github/urmidedhia/Synapse-Learning-Period/blob/NLP-week-3/Tasksentiment_rating_rnn_Urmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''

train = pd.read_csv('D:\\Coding\\Hackathons\\Codeshastra-checkmate\\train_set.csv')
test = pd.read_csv('D:\\Coding\\Hackathons\\Codeshastra-checkmate\\test_set.csv')


# reviews

In [4]:
train.nine_box_category.unique()

array(["Category 1: 'Risk' (Low performance, Low potential)",
       "Category 2: 'Average performer' (Moderate performance, Low potential)",
       "Category 3: 'Solid Performer' (High performance, Low potential)",
       "Category 4: 'Inconsistent Player' (Low performance, Moderate potential)",
       "Category 5: 'Core Player' (Moderate performance, Moderate potential)",
       "Category 6: 'High Performer' (High performance, Moderate potential)",
       "Category 7: 'Potential Gem' (Low performance, High potential)",
       "Category 8: 'High Potential' (Moderate performance, High potential)",
       "Category 9: 'Star' (High performance, High potential)"],
      dtype=object)

In [5]:
def encode_rating(df):
    for i in range(len(df)):
        df.iloc[i, 2] = df.iloc[i, 2][9]
    return df

In [6]:
train_df = encode_rating(train)
train_df = train_df.drop(['id','person_name','adjusted','reviewed'],axis=1)

In [7]:
test.columns

Index(['id', 'person_name', 'nine_box_category', 'feedback', 'updated',
       'reviewed', 'label', 'performance_class', 'potential_class',
       'feedback_clean'],
      dtype='object')

In [8]:
test_df = encode_rating(test)
test_df = test_df.drop(['id','person_name','updated','reviewed','label', 'performance_class', 'potential_class',
       'feedback_clean'],axis=1)

In [9]:
test_df.shape

(225, 2)

In [10]:
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [11]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [12]:
#Split the data into train and split
# X = train_df[['feedback']] + test_df[['feedback']]
# y = train_df[['nine_box_category']] + test_df[['nine_box_category']]
# from sklearn.model_selection import train_test_split
train_reviews, test_reviews, train_sentiment, test_sentiment = train_df[['feedback']],test_df[['feedback']],train_df[['nine_box_category']],test_df[['nine_box_category']]

train_sentiment = pd.get_dummies(train_sentiment['nine_box_category']).values
test_sentiment = pd.get_dummies(test_sentiment['nine_box_category']).values

In [13]:
# # convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# # Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# # In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0
# for df in [train_sentiment, test_sentiment]:
#   df['sentiment'] = df['sentiment'].replace(['positive'],1)
#   df['sentiment'] = df['sentiment'].replace(['negative'],0)

# test_sentiment

In [14]:
print(train_reviews.shape,test_reviews.shape)
print(train_sentiment.shape,test_sentiment.shape)

(878, 1) (225, 1)
(878, 9) (225, 9)


In [15]:
train_reviews


,feedback
0,John has not progressed in his position. He is...
1,"I'm always impressed by Braydon, since he perf..."
2,Celine is one of the beat employees we have to...
3,Eloise Foster is a follower rather than a lead...
4,Daniel Hill has come far and has it in him to ...
...,...
873,Finley Grey completes tasks on time and follow...
874,"Andy was a very inconsistent player, he would ..."
875,Jude May is the person who needs to concentrat...
876,Madison is a very strong worker that consisten...


In [16]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    tokenized = [[word.lower_ for word in encoder(text)] for text in text_seqs]
    return tokenized
    #complete this function that lowers and tokenizes the reviews


train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['feedback'])
train_reviews[['feedback','Tokenized_Review']][:10]

,feedback,Tokenized_Review
0,John has not progressed in his position. He is...,"[john, has, not, progressed, in, his, position..."
1,"I'm always impressed by Braydon, since he perf...","[i, 'm, always, impressed, by, braydon, ,, sin..."
2,Celine is one of the beat employees we have to...,"[celine, is, one, of, the, beat, employees, we..."
3,Eloise Foster is a follower rather than a lead...,"[eloise, foster, is, a, follower, rather, than..."
4,Daniel Hill has come far and has it in him to ...,"[daniel, hill, has, come, far, and, has, it, i..."
5,Kyle is working toward gaining proficiency. De...,"[kyle, is, working, toward, gaining, proficien..."
6,Theodor Peters is an excellent player with a h...,"[theodor, peters, is, an, excellent, player, w..."
7,Requires additional scope[e. Willingness to se...,"[requires, additional, scope[e, ., willingness..."
8,Alex is a stable and consistent working on the...,"[alex, is, a, stable, and, consistent, working..."
9,she tries hard but she is in the wrong positio...,"[she, tries, hard, but, she, is, in, the, wron..."


In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
train_reviews['Tokenized_Review'] = train_reviews['Tokenized_Review'].apply(lambda x: [word for word in x if word not in (stop)])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Urmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
train_reviews

,feedback,Tokenized_Review
0,John has not progressed in his position. He is...,"[john, progressed, position, ., continuously, ..."
1,"I'm always impressed by Braydon, since he perf...","['m, always, impressed, braydon, ,, since, per..."
2,Celine is one of the beat employees we have to...,"[celine, one, beat, employees, date, ., comes,..."
3,Eloise Foster is a follower rather than a lead...,"[eloise, foster, follower, rather, leader, ., ..."
4,Daniel Hill has come far and has it in him to ...,"[daniel, hill, come, far, real, star, player, ..."
...,...,...
873,Finley Grey completes tasks on time and follow...,"[finley, grey, completes, tasks, time, follows..."
874,"Andy was a very inconsistent player, he would ...","[andy, inconsistent, player, ,, would, day, ev..."
875,Jude May is the person who needs to concentrat...,"[jude, may, person, needs, concentrate, work, ..."
876,Madison is a very strong worker that consisten...,"[madison, strong, worker, consistently, produc..."


In [19]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for text in token_seqs:
      for token in text:
        if token in token_counts:
          token_counts[token] += 1
        else:
          token_counts[token] = 1
  
    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon = [token for token, _ in token_counts.items() if token_counts[token] >= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {token:index+2 for index, token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'john': 2, 'progressed': 3, 'position': 4, '.': 5, 'continuously': 6, 'late': 7, ',': 8, 'leaves': 9, 'early': 10, 'takes': 11, 'many': 12, 'breaks': 13, 'throughout': 14, 'day': 15, 'calls': 16, 'least': 17, 'every': 18, 'week': 19, "'s": 20, 'always': 21, 'fridays': 22, 'performance': 23, 'significantly': 24, 'declined': 25, 'suggestion': 26, 'suitable': 27, "'m": 28, 'impressed': 29, 'braydon': 30, 'since': 31, 'performs': 32, 'like': 33, 'ten': 34, 'years': 35, 'experience': 36, 'behind': 37, 'even': 38, 'though': 39, 'got': 40, 'five': 41, 'real': 42, 'eye': 43, 'clients': 44, 'want': 45, 'see': 46, 'reports': 47, 'great': 48, 'sense': 49, 'intuition': 50, 'next': 51, 'steps': 52, 'client': 53, 'project': 54, 'going': 55, 'much': 56, 'star': 57, 'however': 58, "n't": 59, 'shown': 60, 'interest': 61, 'mentoring': 62, 'younger': 63, 'colleagues': 64, 'team': 65, 'player': 66, 'thus': 67, 'believe': 68, 'room': 69, 'promotion': 70, 'firm': 71, 'celine': 72, 'one': 73, 'beat': 74, 'e

In [20]:
len(lexicon)

3795

In [ ]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in text] for text in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

In [ ]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = numpy.array([numpy.bincount(numpy.array(seq), minlength = matrix_length) 
                            for seq in idx_seqs])
    return bow_seqs
    

bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_seq_len = max([len(seq) for seq in train_reviews['Review_Idxs']])
def pad_idx_seqs(idx_seqs, length):
    
    #find the biggest review's length and save it in the variable below 
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    padded_seq = pad_sequences(sequences = idx_seqs, maxlen = length)
    return padded_seq

train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'],max_seq_len)

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

In [ ]:
len(train_sentiment)

In [ ]:
'''Create the model'''
# train_sentiment = pd.get_dummies(train_sentiment['nine_box_category']).values
# train_senti_tensor = tf.convert_to_tensor(train_sentiment.values, dtype = np.int64)
# y_train = to_categorical(y_train, 3)
# y_test = to_categorical(y_test, 3)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping


def create_rnn_model(MAX_NB_WORDS, EMBEDDING_DIM, length):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.

    # define the model


    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=length))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(500, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(9, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    epochs = 20
    batch_size = 64
    model.summary()
    history = model.fit(train_padded_idxs, train_sentiment, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

#     input_layer = Input(shape=(None,))
    
#     # Layer 2
#     embedding_layer = Embedding(input_dim=n_input_nodes, output_dim=n_embedding_nodes, mask_zero=True)(input_layer)

#     # Layer 3
#     gru_layer = GRU(units = n_hidden_nodes)(embedding_layer)

#     # Layer 4
#     output_layer = Dense(units = 9)(gru_layer)

#     #Specify which layers are input and output, compile model with loss and optimization functions
#     model = Model(inputs=[input_layer], outputs=output_layer)
#     model.compile(loss="mean_squared_error", optimizer='adam')
    return model



In [ ]:
model = create_rnn_model(2000,100,max_seq_len)

In [ ]:
# '''
# Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
# You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
# Hint: tf.convert_to_tensor
# batch_size=20, epochs=5
# '''
# train_senti_tensor = tf.convert_to_tensor(train_sentiment.values, dtype = np.int64)
# rnn_model.fit(x=train_padded_idxs, y=train_senti_tensor, batch_size=20, epochs=5)

In [ ]:
'''Put test reviews in padded matrix just how we did for train_reviews'''
test_reviews['Tokenized_Review'] = text_to_tokens(test_reviews['feedback'])
# lexicon_test = make_lexicon(token_seqs=test_reviews['Tokenized_Review'], min_freq=1)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'], lexicon=lexicon)
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'],max_seq_len)

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

In [ ]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(model.predict(test_padded_idxs)[:,0]).astype(int)

In [ ]:
test_reviews
import sklearn
accr = model.evaluate(test_padded_idxs,test_sentiment)
# score = sklearn.metrics.accuracy_score(y_true = test_sentiment, y_pred = test_reviews['RNN_Pred_Rating'])


In [ ]:
# '''Evaluate the model with R^2'''

# # print the r2 score
# from sklearn.metrics import r2_score 
# final_score = r2_score(y_true = test_sentiment, y_pred = test_reviews['RNN_Pred_Rating'])
# print(final_score)

In [ ]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])